In [2]:
import sys
import os
import torch
import glob
import pandas as pd
import numpy as np
import pingouin as pg
from age_prediction.models.\
    efficientnet_pytorch_3d import EfficientNet3D as EfNetB0
from age_prediction.metrics import MSE, MAE
from age_prediction.trainer import ModuleTrainer
from age_prediction.dataloader import MyDataLoader

In [8]:
def check_state_dict(ck, md):
    if ck.keys() == md.keys():
        return ''
    else:
        split_key = list(ck.keys())[0].split(
            list(md.keys())[0])[0]
        return split_key

def correct_state_dict(ck, split_key):
    ck2 = {}
    for k in ck.keys():
        ck2[k.split(split_key)[1]] = ck[k]
    return ck2

def MAE_pred(y_pred, y_true):
    mae = torch.nn.L1Loss(reduction='mean')(y_pred, y_true).cpu().detach().numpy().item()
    return mae

def MSE_pred(y_pred, y_true):
    mse = torch.nn.MSELoss()(y_pred, y_true).cpu().detach().numpy().item()
    return mse

def delta_pred(y_pred, y_true):
    delta = y_pred - y_true
    return np.mean(delta)

def pearson(y_pred, y_true):
    return pg.corr(y_pred, y_true)['r'][0]


In [9]:
def get_snapshot_results(side, snapshot, dropout_rate, testfile):
    # Load effNet3D B0
    model = EfNetB0.from_name("efficientnet-b0",
                            override_params={
                                'num_classes': 1,
                                'dropout_rate': dropout_rate
                            },
                            in_channels=1,
                            )
    device = torch.device('cpu')
    model = model.to(device)

    optimizer = torch.optim.RMSprop(model.parameters(),
                                lr=.256, alpha=0.9,
                                eps=1e-08, momentum=0.9,
                                weight_decay=0)

    print('Loading model from {}'.format(snapshot))
    checkpoint = torch.load(snapshot, map_location=device)
    split_key = check_state_dict(checkpoint['state_dict'], model.state_dict())
    print(list(checkpoint['state_dict'])[0])
    if split_key is not None:
        checkpoint['state_dict'] = correct_state_dict(checkpoint['state_dict'], split_key)

    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    _loss = checkpoint['loss']
    _val_loss = checkpoint['val_loss']
    print("Snapshot trained for {} epochs. \
        Loss: {} and Val loss {}".format(epoch, _loss, _val_loss))

    loss = torch.nn.L1Loss(reduction='mean')
    metrics = [MSE()]
    
    # Predict
    trainer = ModuleTrainer(model.to(device))

    trainer.compile(loss=loss,
                    optimizer=optimizer,
                    metrics=metrics)
    
    dataloader = MyDataLoader(database='../datasets',
                          csv_data='database_split',
                          side=side,
                          batch=512,
                          data_aug=False,
                          test_file=testfile
                          )
    dataloader.prepare_data('test_label')
    dataloader.setup('test_label')
    
    print("Test size", len(dataloader.test.inputs[0]))


    pred = trainer.predict_loader(dataloader.testlabel_dataloader(),
                                  cuda_device=False)
    
    imgs = dataloader.testpath
    imgs = [img.split("/")[-1].split(side)[0] for img in imgs]
    
    preds = pd.concat([pd.DataFrame(imgs),
                       pd.DataFrame(dataloader.testlabel),
                       pd.DataFrame(pred.detach().cpu().numpy())], axis=1)
    preds.columns = ['Name', 'True', 'Pred']
    
    return preds

def metrics_pipeline(preds):
    print("MAE", MAE_pred(torch.from_numpy(preds['Pred'].values), torch.from_numpy(preds['True'].values)))
    print("MSE", MSE_pred(torch.from_numpy(preds['Pred'].values), torch.from_numpy(preds['True'].values)))
    print("Delta", delta_pred(preds['Pred'], preds['True']))
    print("Pearson", pearson(preds['Pred'], preds['True']))

In [15]:
side = "_L"
for ckpt in glob.glob('outputs/ckpt*_best*'):
    if "e-5" not in ckpt:
        dropout_rate = float(ckpt.split("_dp")[-1].split("_")[0])
        print("###"*5)
        pred = get_snapshot_results(side, ckpt, dropout_rate, 'val_0-70.csv')
        metrics_pipeline(pred)

###############
Dropout 0.6
Loading model from outputs/ckpt_27-04-2021_age_[0-70]_RMS_wd_0_L_dp0.6_model_best.pth.tar
module._conv_stem.weight
Snapshot trained for 203 epochs.         Loss: 30.867122573401094 and Val loss 31.042200088500977
Preparing data
Setup data
Test size 67
MAE 4.337871380706332
MSE 31.042237827168353
Delta -0.2838427301663072
Pearson 0.9364995535561179
###############
Dropout 0.2
Loading model from outputs/ckpt_25-04-2021_age_[0-70]_RMS_wd_0_L_dp0.2_model_best.pth.tar
module._conv_stem.weight
Snapshot trained for 218 epochs.         Loss: 22.58754856854501 and Val loss 26.08509063720703
Preparing data
Setup data
Test size 67
MAE 3.563694619705428
MSE 26.085099133817188
Delta 0.2178284488507171
Pearson 0.9471203416097504
###############
Dropout 0.5
Loading model from outputs/ckpt_27-04-2021_age_[0-70]_RMS_wd_0_L_dp0.5_model_best.pth.tar
module._conv_stem.weight
Snapshot trained for 179 epochs.         Loss: 23.185775490484293 and Val loss 30.183307647705078
Prepar

In [14]:
side = "_L"
for ckpt in glob.glob('outputs/ckpt*_best*'):
    if "e-5" not in ckpt:
        dropout_rate = float(ckpt.split("_dp")[-1].split("_")[0])
        print("###"*5)
        pred = get_snapshot_results(side, ckpt, dropout_rate, 'test_0-70.csv')
        metrics_pipeline(pred)

###############
Dropout 0.6
Loading model from outputs/ckpt_27-04-2021_age_[0-70]_RMS_wd_0_L_dp0.6_model_best.pth.tar
module._conv_stem.weight
Snapshot trained for 203 epochs.         Loss: 30.867122573401094 and Val loss 31.042200088500977
Preparing data
Setup data
Test size 67
MAE 5.33826094214596
MSE 48.72745083675871
Delta -1.0490870324889225
Pearson 0.9006745698169084
###############
Dropout 0.2
Loading model from outputs/ckpt_25-04-2021_age_[0-70]_RMS_wd_0_L_dp0.2_model_best.pth.tar
module._conv_stem.weight
Snapshot trained for 218 epochs.         Loss: 22.58754856854501 and Val loss 26.08509063720703
Preparing data
Setup data
Test size 67
MAE 5.0888911483536905
MSE 56.416065669994246
Delta -0.5379316050970729
Pearson 0.8819676444759508
###############
Dropout 0.5
Loading model from outputs/ckpt_27-04-2021_age_[0-70]_RMS_wd_0_L_dp0.5_model_best.pth.tar
module._conv_stem.weight
Snapshot trained for 179 epochs.         Loss: 23.185775490484293 and Val loss 30.183307647705078
Prepar

In [12]:
side = "_L"
for ckpt in glob.glob('outputs/ckpt*'):
    if "best" not in ckpt:
        dropout_rate = float(ckpt.split("_dp")[-1].split(".pth")[0])
        print("###"*5)
        pred = get_snapshot_results(side, ckpt, dropout_rate, 'val_0-70.csv')
        metrics_pipeline(pred)

###############
Dropout 0.6
Loading model from outputs/ckpt_28-04-2021_age_[0-70]_RMS_wd_1e-5_L_dp0.6.pth.tar
module._conv_stem.weight
Snapshot trained for 2 epochs.         Loss: 2966.0897654516457 and Val loss 2103.302490234375
Preparing data
Setup data
Test size 67
MAE 43.03559482093177
MSE 2103.3025255712078
Delta -43.03559482093177
Pearson 0.07848531504462533
###############
Dropout 0.5
Loading model from outputs/ckpt_28-04-2021_age_[0-70]_RMS_wd_1e-5_L_dp0.5.pth.tar
module._conv_stem.weight
Snapshot trained for 400 epochs.         Loss: 235.08455294095552 and Val loss 333.3091125488281
Preparing data
Setup data
Test size 67
MAE 16.687722876249854
MSE 333.30921694386114
Delta 9.059200469059732
Pearson 0.003073025316861669
###############
Dropout 0.3
Loading model from outputs/ckpt_26-04-2021_age_[0-70]_RMS_wd_0_L_dp0.3.pth.tar
module._conv_stem.weight
Snapshot trained for 400 epochs.         Loss: 5.4130309731297235 and Val loss 28.97721290588379
Preparing data
Setup data
Test siz

In [13]:
side = "_L"
for ckpt in glob.glob('outputs/ckpt*'):
    if "best" not in ckpt:
        dropout_rate = float(ckpt.split("_dp")[-1].split(".pth")[0])
        print("###"*5)
        pred = get_snapshot_results(side, ckpt, dropout_rate, 'test_0-70.csv')
        metrics_pipeline(pred)

###############
Dropout 0.6
Loading model from outputs/ckpt_28-04-2021_age_[0-70]_RMS_wd_1e-5_L_dp0.6.pth.tar
module._conv_stem.weight
Snapshot trained for 3 epochs.         Loss: 2959.4410523183246 and Val loss 2096.1787109375
Preparing data
Setup data
Test size 67
MAE 49.963794775009156
MSE 2743.2245565202393
Delta -49.963794775009156
Pearson 0.10231184494553737
###############
Dropout 0.5
Loading model from outputs/ckpt_28-04-2021_age_[0-70]_RMS_wd_1e-5_L_dp0.5.pth.tar
module._conv_stem.weight
Snapshot trained for 400 epochs.         Loss: 235.08455294095552 and Val loss 333.3091125488281
Preparing data
Setup data
Test size 67
MAE 13.962019956218663
MSE 251.03871208045237
Delta 2.0481556929403277
Pearson -0.03579344977284786
###############
Dropout 0.3
Loading model from outputs/ckpt_26-04-2021_age_[0-70]_RMS_wd_0_L_dp0.3.pth.tar
module._conv_stem.weight
Snapshot trained for 400 epochs.         Loss: 5.4130309731297235 and Val loss 28.97721290588379
Preparing data
Setup data
Test si